In [2]:
#! -*- coding:utf-8 -*-
import re, os, json, codecs, gc
import numpy as np
import pandas as pd
from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam

#train_label = pd.read_csv('data_car/test.csv')
#train_df = pd.merge(train_df, train_label, on='id')
train_df = pd.read_csv('data_car/train.csv')
test_df = pd.read_csv('data_car/test.csv')
#test_lines = codecs.open('data_car/test.csv').readlines()[1:]
train_df['tb'] = train_df['title'].replace(' ', '') + train_df['content'].replace(' ', '')
test_df['tb'] = test_df['title'].replace(' ', '') + test_df['content'].replace(' ', '')
train_df['tb'] =train_df['tb'].astype('str')
test_df['tb'] = test_df['tb'].astype('str')
#! -*- coding:utf-8 -*-
import re, os, json, codecs, gc
import numpy as np
import pandas as pd
from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import tokenization
from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # 按照PCI_BUS_ID顺序从0开始排列GPU设备 

maxlen = 512
config_path = '/home/jovyan/work/data/chinese_L-12_H-768_A-12/bert_config.json'
# checkpoint_path = '/export/home/liuyuzhong/kaggle/bert/chinese_L-12_H-768_A-12/bert_model.ckpt'
checkpoint_path = '/home/jovyan/work/data/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/home/jovyan/work/data/chinese_L-12_H-768_A-12/vocab.txt'

token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)
tokenizer1 = tokenization.FullTokenizer(vocab_file=dict_path, do_lower_case=True)
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

def dategenenrator(data, batch_size=1, shuffle=True):
    X1, X2, Y = [], [], []
    idxs = list(range(len(data)))
    if shuffle:
        np.random.shuffle(idxs)
    for i in idxs:
        d = data[i]
        text = d[0][:maxlen-2]
        x1,x2=tokenizer.encode(first=text)
        y = d[1]
        X1.append(x1)
        X2.append(x2)
        Y.append([y])
    X1 = seq_padding(X1)
    X2 = seq_padding(X2)
    Y = seq_padding(Y)
    Y = np.array([np.argmax(x) for x in Y])
    return [X1,X2],Y

def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    all_mask=[]
    longer = 0
    for i in range(example.shape[0]):
      tokens_a = tokenizer.tokenize(example[i])
      if len(tokens_a)>max_seq_length:
        #tokens_a = tokens_a[:int(max_seq_length/2)] +tokens_a[-int(max_seq_length/2):]
        
        tokens_a = tokens_a[:max_seq_length]        
        longer += 1
        mask=[1]*(max_seq_length+2)
      else:
        mask=[1]*(len(tokens_a)+2)+[0]*(max_seq_length-len(tokens_a))
      one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
      all_tokens.append(one_token)
      all_mask.append(mask)
    print(longer)
    return np.array(all_tokens),np.array(all_mask)

class data_generator:
    def __init__(self, data, batch_size=1, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            
            if self.shuffle:
                np.random.shuffle(idxs)
            
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen-2]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y[:, 0, :]
                    [X1, X2, Y] = [], [], []

from keras.metrics import top_k_categorical_accuracy
def acc_top2(y_true, y_pred):
    
    return top_k_categorical_accuracy(y_true, y_pred, k=2)
                    
def build_bert(nclass):
    bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

    for l in bert_model.layers:
        l.trainable = True

    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))

    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x)
    p = Dense(1, activation='sigmoid')(x)

    model = Model([x1_in, x2_in], p)
    model = multi_gpu_model(model,gpus=[5,6])
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(1e-5),
                  metrics=['accuracy', acc_top2])
    print(model.summary())
    return model
from keras.utils import to_categorical
#os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
DATA_LIST = []
for data_row in train_df.iloc[:].itertuples():
    DATA_LIST.append((data_row.tb, to_categorical(data_row.flag, 2)))
DATA_LIST = np.array(DATA_LIST)

df_pre = pd.read_excel('data_car/pesudodata.xlsx')
df_pre = df_pre[pd.isna(df_pre.pre)==False]
df_pre['flag'] = df_pre['pre']
df_pre['tb'] = df_pre['title'].replace(' ', '') + df_pre['content'].replace(' ', '')
df_pre['tb'] = df_pre['tb'].astype('str')
DATA_LIST_PRE = []
for data_row in df_pre.iloc[:].itertuples():
    DATA_LIST_PRE.append((data_row.tb, to_categorical(data_row.flag, 2)))
DATA_LIST_PRE = np.array(DATA_LIST_PRE)

DATA_LIST_TEST = []
for data_row in test_df.iloc[:].itertuples():
    DATA_LIST_TEST.append((data_row.tb, to_categorical(0, 2)))
DATA_LIST_TEST = np.array(DATA_LIST_TEST)
def run_cv(nfold, data, data_label, data_test):
        
    kf = KFold(n_splits=nfold, shuffle=True, random_state=520).split(data)
    train_model_pred = np.zeros((len(data), 1))
    test_model_pred = np.zeros((len(data_test), 1))
    #oof_train = np.zeros((len(train),1))
    #oof_test = np.zeros((len(test_df),1))
    for i, (train_fold, test_fold) in enumerate(kf):
        X_train, X_valid, = data[train_fold, :], data[test_fold, :]
        #X_train = np.concatenate((X_train, DATA_LIST_PRE))
        model = build_bert(2)
        early_stopping = EarlyStopping(monitor='val_acc', patience=3)
        plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
        checkpoint = ModelCheckpoint('baseline2/base_roberta/' + str(i) + '.hdf5', monitor='val_acc', 
                                         verbose=2, save_best_only=True, mode='max',save_weights_only=True)
        
        train_D,train_Y= dategenenrator(X_train, shuffle=True)
        valid_D,valid_Y = dategenenrator(X_valid, shuffle=True)
        test_D,test_Y = dategenenrator(data_test, shuffle=False)
        
        
        model.fit(train_D,train_Y,\
               validation_data = (valid_D,valid_Y),
               callbacks=[early_stopping, plateau, checkpoint],batch_size=1,epochs=5,shuffle=True)
        
        '''model.fit_generator(
            train_D.__iter__(),
            steps_per_epoch=len(train_D),
            epochs=5,
            validation_data=valid_D.__iter__(),
            validation_steps=len(valid_D),
            callbacks=[early_stopping, plateau, checkpoint],
        )'''
        
        #model.load_weights('bert_dump_base/' + str(i) + '.hdf5')
        
        # return model
        train_model_pred[test_fold, :] =  model.predict(valid_D,verbose=1)
        test_pred_temp =  model.predict(test_D,verbose=1)       
        test_model_pred += test_pred_temp
        #oof_train[idx_test] = test_pred_temp
        x_pre = model.predict(test_D)
        test_df['label'] = x_pre
        test_df.to_csv('baseline2/base_roberta/1029'+str(i)+'.csv',index=False) 
        #oof_test += x_pre/skf.n_splits
        del model; gc.collect()
        K.clear_session()
        
        # break
        
    return train_model_pred, test_model_pred
train_model_pred, test_model_pred = run_cv(10, DATA_LIST, None, DATA_LIST_TEST)

ValueError: setting an array element with a sequence

In [2]:
def predictvalue(oof_test):
    listvalue = []
    for i in range(len(oof_test)):
        if oof_test[i] > 0.5:
            listvalue.append(1)
        else:
            listvalue.append(0)
    return listvalue

prevalue = predictvalue(test_model_pred/10)
test_df['label'] = prevalue
test_df[['id', 'label']].to_csv('baseline2/base_roberta/baseline3_1029.csv', index=None)

NameError: name 'test_model_pred' is not defined

In [2]:
testpre = pd.DataFrame({'id':test_df['id'],'label':test_df['labelpre']})
testpre.to_csv('testpredictcar_pesudo_1024.csv',index=False)

In [4]:
df['pre']=oof_train
df.to_csv('trainpredictcar_pesudo_1024.csv',index=False)
from sklearn.metrics import roc_curve, precision_recall_curve
def threshold_search(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result 

result = threshold_search(df.flag.values,oof_train)
print(result)

{'threshold': 0.5033168196678162, 'f1': 0.8451261751608115}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
